In [1]:
%load_ext autoreload
%autoreload 2

import os
import pymupdf
import json
import sys


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if repo_root not in sys.path:
    sys.path.append(repo_root)

from src.text import extract_words, process_documents
from src.bounding_box import cluster_drawings
from src.utils import is_digitally_born, classify_text_density


In [2]:
filename ="Berichte_NAB 10-025_Kurzarbeitsprogram Geothermiebohrung Schlattingen.PDF"

pdf_path = os.path.join(repo_root, "data/NAB", filename)
out_dir = os.path.join(repo_root, "data/test", os.path.splitext(filename)[0])


In [3]:
def classify_digital(page):
    if page.get_images():
        return " image"

    drawings = page.get_drawings()
    if drawings:
        clustered_boxes = cluster_drawings(drawings)

        for cluster in clustered_boxes: #filter out small bboxes
            if cluster.get_area() > 100000:
                return " drawing"
    return " text"

In [4]:
##classify pdf based on image, drawing and text
def classify_pages(doc):
    results ={}
    for page_index, page in enumerate(doc):
        page_number = page_index + 1
        page_size = (page.rect.width, page.rect.height)
        
        digitally_born = is_digitally_born(doc[page_index])

        if digitally_born:
            page_type = classify_digital(page)
        
        else:
            words = extract_words(page, page_number)
            page_type = classify_text_density(words, page_size)
        
        results[page_number] = { "page_type":page_type,
                                "digitally_born": digitally_born}
            
    return results


In [ ]:
predictions = process_documents(pdf_path)
with open(os.path.join(repo_root,"data/predictions.json"), 'w', encoding='utf-8') as json_file:
        json.dump(predictions, json_file, indent=4, ensure_ascii=False)

TypeError: process_documents() takes 1 positional argument but 2 were given